# Model V1

In [ ]:
import pathlib
import tensorflow as tf
import pandas as pd

from bcd.model.factory import ModelFactoryV1
from bcd.model.experiment import Experiment
from bcd.model.repo import ModelRepo
pd.set_option('display.max_rows',999)


## Configuration

In [ ]:
# Experiment Parameters
%env "WANDB_NOTEBOOK_NAME" "01_model_V1.ipynb"
project = "BCD_V1"
full_dataset = False

# Model Parameters
force = False  # Whether to retrain if the model and weights already exist from a prior training session.
metrics = ['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
loss = "binary_crossentropy"
activation = "sigmoid"
epochs = 100
learning_rate = 1e-4
num_classes = 2

# Dataset params
dataset = "CBIS-DDSM" if full_dataset else "CBIS-DDSM-10"
batch_size = 64 if full_dataset else 32
input_shape = (224,224,3)
output_shape = 1
train_dir = pathlib.Path("data/image/1_final/training/training/").with_suffix('') if full_dataset else pathlib.Path("data/image/1_final/training_10/training/").with_suffix('') 
test_dir = pathlib.Path("data/image/1_final/test/test/").with_suffix('')

# Early stop parameters 
es_min_delta = 0.0001
es_monitor = "val_loss"  # Monitor validation loss for early stopping
es_patience = 8  # The number of epochs for which lack of improvement is tolerated 
es_restore_best_weights = True  # Returns the best weights rather than the weights at the last epoch.
es_verbose = 1

# Reduce LR on Plateau Parameters
rlr_monitor = "val_loss"
rlr_factor = 0.5
rlr_patience = 3
rlr_verbose = 1
rlr_mode = "auto"
rlr_min_delta = 1e-4
rlr_min_lr=1e-10



## Experiment Config

In [ ]:
config = {
    "project": project,        
    "dataset": dataset,
    "num_classes": num_classes,
    "batch_size": batch_size,
    "epochs": epochs,    
    "learning_rate": learning_rate,
    "loss": loss,    
    "early_stop_min_delta": es_min_delta,
    "early_stop_monitor": es_monitor,
    "early_stop_patience": es_patience,
    "early_stop_restore_best_weights": es_restore_best_weights,
    "early_stop_verbose": es_verbose,
    "rlr_monitor": rlr_monitor,
    "rlr_factor": rlr_factor,
    "rlr_patience": rlr_patience,
    "rlr_verbose": rlr_verbose,
    "rlr_mode": rlr_mode,
    "rlr_min_delta": rlr_min_delta,
    "rlr_min_lr": rlr_min_lr
    
}

## Load Data

In [ ]:
# Training DataSet (10%)
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    seed=123,
    batch_size=batch_size)

# Validation DataSet (10%)
val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    seed=123,
    batch_size=batch_size)

# Test Set
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True)

## Callbacks

In [ ]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor=es_monitor, 
                                                       min_delta=es_min_delta,
                                                       patience=es_patience, 
                                                       restore_best_weights=es_restore_best_weights,
                                                       verbose=es_verbose)

reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor=rlr_monitor,
                                                          factor=rlr_factor,
                                                          patience=rlr_patience,
                                                          verbose=rlr_verbose,
                                                          mode=rlr_mode,
                                                          min_delta=rlr_min_delta,
                                                          min_lr=rlr_min_lr)
callbacks = [early_stop_callback, reduce_lr_callback]

## Dependencies

In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
repo = ModelRepo()

## DenseNet Experiment

In [ ]:
# Obtain the DenseNet model
factory = ModelFactoryV1(input_shape=input_shape, output_shape=output_shape, activation=activation)
densenet = factory.create_densenet()
densenet.summary()

# Create and run the experiment.
densenet_experiment = Experiment(model=densenet, config=config, repo=repo, optimizer=optimizer, callbacks=callbacks, metrics=metrics, force=force)
densenet_experiment.run(train_ds=train_ds, val_ds=val_ds)
densenet_experiment.classification_report(data=val_ds)
densenet_experiment.plot_confusion_matrix(data=val_ds)
densenet_experiment.evaluate(data=test_ds)

## ResNet Experiment

In [ ]:
# Obtain the ResNet model
resnet = factory.create_resnet()
resnet.summary()

# Create and run the experiment.
resnet_experiment = Experiment(model=resnet, config=config, repo=repo, optimizer=optimizer, callbacks=callbacks, metrics=metrics, force=force)
resnet_experiment.run(train_ds=train_ds, val_ds=val_ds)
resnet_experiment.classification_report(data=val_ds)
resnet_experiment.plot_confusion_matrix(data=val_ds)
resnet_experiment.evaluate(data=test_ds)

## Inception Experiment

In [ ]:
# Obtain the ResNet model
inception = factory.create_inception()
inception.summary()

# Create and run the experiment.
inception_experiment = Experiment(model=inception, config=config, repo=repo, optimizer=optimizer, callbacks=callbacks, metrics=metrics, force=force)
inception_experiment.run(train_ds=train_ds, val_ds=val_ds)
inception_experiment.classification_report(data=val_ds)
inception_experiment.plot_confusion_matrix(data=val_ds)
inception_experiment.evaluate(data=test_ds)


## EfficientNet Experiment

In [ ]:
# Obtain the Efficientnet model
efficientnet = factory.create_efficientnet()
efficientnet.summary()

# Create and run the experiment.
efficientnet_experiment = Experiment(model=efficientnet, config=config, repo=repo, optimizer=optimizer, callbacks=callbacks, metrics=metrics, force=force)
efficientnet_experiment.run(train_ds=train_ds, val_ds=val_ds)
efficientnet_experiment.classification_report(data=val_ds)
efficientnet_experiment.plot_confusion_matrix(data=val_ds)
efficientnet_experiment.evaluate(data=test_ds)


## Inception/Resnet Experiment

In [ ]:
# Obtain the Inception_resnet model
inception_resnet = factory.create_inception_resnet()
inception_resnet.summary()

# Create and run the experiment.
inception_resnet_experiment = Experiment(model=inception_resnet, config=config, repo=repo, optimizer=optimizer, callbacks=callbacks, metrics=metrics, force=force)
inception_resnet_experiment.run(train_ds=train_ds, val_ds=val_ds)
inception_resnet_experiment.classification_report(data=val_ds)
inception_resnet_experiment.plot_confusion_matrix(data=val_ds)
inception_resnet_experiment.evaluate(data=test_ds)


## MobileNet Experiment

In [ ]:
# Obtain the Mobilenet model
mobilenet = factory.create_mobilenet()
mobilenet.summary()

# Create and run the experiment.
mobilenet_experiment = Experiment(model=mobilenet, config=config, repo=repo, optimizer=optimizer, callbacks=callbacks, metrics=metrics, force=force)
mobilenet_experiment.run(train_ds=train_ds, val_ds=val_ds)
mobilenet_experiment.classification_report(data=val_ds)
mobilenet_experiment.plot_confusion_matrix(data=val_ds)
mobilenet_experiment.evaluate(data=test_ds)


## Xception Experiment

In [ ]:
# Obtain the Xception model
xception = factory.create_xception()
xception.summary()

# Create and run the experiment.
xception_experiment = Experiment(model=xception, config=config, repo=repo, optimizer=optimizer, callbacks=callbacks, metrics=metrics, force=force)
xception_experiment.run(train_ds=train_ds, val_ds=val_ds)
xception_experiment.classification_report(data=val_ds)
xception_experiment.plot_confusion_matrix(data=val_ds)
xception_experiment.evaluate(data=test_ds)
